In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import time
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator, load_img
import pandas as pd

In [2]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

x_train = x_train / 255.0
x_test = x_test / 255.0

In [4]:
batch_size = 256
epoch = 50
image_size = 8

In [5]:
# Early stopping(patience=5), adam optimizer

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# DEFINE MODEL
# 512 relu
# 10 softmax
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# COMPILE
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

start = time.time()

# TRAIN
# early stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size, epochs=epoch, verbose=1,
                    callbacks=[early_stopping, model_checkpoint])

print('training time: ', time.time() - start)

2022-05-31 15:25:05.386371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 15:25:05.524636: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 15:25:05.525410: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-31 15:25:05.526683: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/50
235/235 [==============================] - 2s 3ms/step - loss: 0.3301 - accuracy: 0.9083
Epoch 2/50
235/235 [==============================] - 1s 2ms/step - loss: 0.1374 - accuracy: 0.9611
Epoch 3/50
235/235 [==============================] - 1s 2ms/step - loss: 0.0908 - accuracy: 0.9739
Epoch 4/50
235/235 [==============================] - 1s 2ms/step - loss: 0.0666 - accuracy: 0.9814
Epoch 5/50
235/235 [==============================] - 1s 2ms/step - loss: 0.0526 - accuracy: 0.9850
Epoch 6/50
235/235 [==============================] - 1s 3ms/step - loss: 0.0410 - accuracy: 0.9883
Epoch 7/50
235/235 [==============================] - 1s 3ms/step - loss: 0.0322 - accuracy: 0.9913
Epoch 8/50
235/235 [==============================] - 1s 3ms/step - loss: 0.0254 - accuracy: 0.9938
Epoch 9/50
235/235 [==============================] - 1s 3ms/step - loss: 0.0200 - accuracy: 0.9954
Epoch 10/50
235/235 [==============================] - 1s 3ms/step - loss: 0.0156 - accuracy: 0.9969

KeyError: 'val_loss'

In [8]:
test_data = pd.read_csv('../input/digit-recognizer/test.csv')
test_data = test_data/255.0
test_data = test_data.values.reshape(-1,28,28,1)

In [9]:
prediction = model.predict(test_data)
prediction = np.argmax(prediction, axis=1)

In [10]:
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sample_submission['Label'] = prediction

In [11]:
sample_submission.tail()

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2


In [12]:
sample_submission.to_csv("submission.csv",index=False)